# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [ ]:
!pip install mediapipe
!pip install groq
!pip install python-dotenv

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from groq import Groq
import os
from dotenv import load_dotenv


In [ ]:
MODELS_PATH = './../model_weights'

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert to RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image is writeable again
    return results


In [ ]:

def draw_styled_landmarks(image, results):
    # Draw hand connections on the black image
    if results.left_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
    if results.right_hand_landmarks:
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2))
        
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             )


In [ ]:

# Extract Keypoints from Landmarks And Concatenate it in One Array
def extract_keypoints(results):
    """
    inputs: Resutls from MediaPipe Model
    output: Concatenated Landmarks
    """
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [ ]:
# Define our Actions
actions = np.array(['drink','eat','friend','goodbye','hello','help','how are you','no','yes','please','sorry','thanks','cry','i','they','you','what','name','teacher','family','happy','love','sad','laugh','neighbor','ok','read','write','school'])
# actions = np.array(['goodbye', 'hello', 'how are you','i','name','no','ok','thanks','what','yes','you'])

# 3. Preprocess Data and Create Labels and Features

In [ ]:
# DataPath of Data
label_map = {label:num for num, label in enumerate(actions)}
label_map

# 4. Load LSTM Model

In [ ]:
# Build a LSTM Model Arch
timesteps=30; features=84
model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(timesteps,features))) # frames * Features
model_lstm.add(LSTM(128, return_sequences=True, activation='relu'))
model_lstm.add(LSTM(64, return_sequences=False, activation='relu'))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(32, activation='relu'))
model_lstm.add(Dense(len(actions), activation='softmax'))
model_lstm.summary()

In [ ]:

model_lstm.load_weights(os.path.join(MODELS_PATH,'LSTM29.h5'))

# 5. LLM Sentence Formation 

In [ ]:
load_dotenv()

API_KEY = os.getenv('API_TOKEN')

client = Groq(api_key=API_KEY)

In [ ]:

def form_sentence_using_llm(scrambledWords):
    
    contentMsg = f"""
    Task: Rearrange the following scrambled words into a coherent sentence and not complete the sentence and give only one answer only. it is okay even if the sentence consist of one word only 
    Example 1:
    Scrambled Words: [love - I - dogs]
    Complete the sentence:
    I love dogs.
    Example 2:
    Scrambled Words: [beautiful - is - today - weather - the]
    Complete the sentence:
    The weather is beautiful today.
    Example 3:
    Scrambled Words: [eat - dinner - we - can - together]
    Complete the sentence:
    We can eat dinner together.
    Example 4:
    Scrambled Words: [ok]
    Complete the sentence:
    Okay.
    New Instance:
    Scrambled Words: [{scrambledWords.replace(' ', ' - ')}]
    Complete the sentence:
    """

    completion = client.chat.completions.create(
        model="mixtral-8x7b-32768",
        messages=[
            {
                "role": "user",
                "content": contentMsg
            }
        ],
        temperature=0.08,
        max_tokens=1024,
        top_p=1,
        stream=True,
        stop=None,
    )
    formed_sentence = []
    for chunk in completion:
        if chunk.choices[0].delta.content not in [None,'']:
            formed_sentence.append(chunk.choices[0].delta.content)
    return ''.join(formed_sentence)

In [ ]:
form_sentence_using_llm('what you name')

# 6. Real Time with skeleton

In [ ]:

sequence = []
sentence = []
predictions = []
formed_sentance = ''
threshold = 0.9

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = mediapipe_detection(frame, holistic)
        black_image = np.zeros(frame.shape, dtype=np.uint8)
        draw_styled_landmarks(black_image, results)
        if results.left_hand_landmarks or results.right_hand_landmarks:
        # Create a black image

            keypoints = extract_keypoints(results)

            lh_rh = keypoints
            # Remove z Axis From Landmarks
            for z in range(2, lh_rh.shape[0], 3):
                lh_rh[z] = None
            # Remove NaN Data
            lh_rh = lh_rh[np.logical_not(np.isnan(lh_rh))]
            sequence.append(lh_rh)
            sequence = sequence[-30:]

            if len(sequence) == 30:
                # Convert sequence to numpy array
                sequence_array = np.array(sequence)

                # Reshape sequence_array to match model input shape
                sequence_array = np.expand_dims(sequence_array, axis=0)  # Add batch dimension
                sequence_array = sequence_array.reshape((1, 30, -1))  # Reshape to (1, 30, features)
                res = model_lstm.predict(sequence_array)[0]
                # print("res", res)
                predictions.append(np.argmax(res))

                if np.unique(predictions[-15:])[0] == np.argmax(res):
                    if res[np.argmax(res)] > threshold:
                        if len(sentence) > 0:
                            if actions[np.argmax(res)] != sentence[-1]:
                                sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

                if len(sentence) > 5:
                    sentence = sentence[-5:]

        # black_image = prob_viz(res, actions, black_image, colors)

        # Get frame height
        # sequence = []
        frame_height = black_image.shape[0]
        formed_sentance = formed_sentance.split('.')[0]
        if len(formed_sentance) == 1:
            ''.join(formed_sentance)
        else:
            ' '.join(formed_sentance)
        # Draw the rectangle at the bottom of the screen
        cv2.rectangle(black_image, (0, frame_height - 40), (640, frame_height), (245, 117, 16), -1)
        cv2.putText(black_image, ' '.join(sentence), (3, frame_height - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.rectangle(black_image, (0, frame_height - 80), (640, frame_height - 40), (117, 245, 16), -1)
        cv2.putText(black_image,formed_sentance, (10, frame_height - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)


        cv2.imshow('OpenCV Feed', black_image)

        key = cv2.waitKey(10)
        if key & 0xFF == ord('q'):
            break
        elif key & 0xFF == ord('s'):
            formed_sentance = form_sentence_using_llm(''.join(sentence))
            print(formed_sentance)
            sentence = []
        elif key & 0xFF == ord('d'):
            sequence = []
            sentence = []
            predictions = []
            formed_sentance = ''
            

    cap.release()
    cv2.destroyAllWindows()
